In [47]:
import pandas as pd
pd.options.mode.chained_assignment = None

In [48]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import mean_squared_error

In [49]:
train_df = pd.read_parquet('./data/fhv_tripdata_2021-01.parquet')
test_df = pd.read_parquet('./data/fhv_tripdata_2021-02.parquet')

### Question 1

In [50]:
print(f"Answer: {len(train_df)}")

Answer: 1154112


In [57]:
def _transform(df):
    answers_dict = {}
    init_size_df = len(df)
    df['duration'] = df.dropOff_datetime - df.pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)
    answers_dict['Q2 avg_duration'] = df['duration'].mean()
    df = df[(df.duration >= 1) & (df.duration <= 60)]
    df.PUlocationID.fillna(-1, inplace=True)
    df.DOlocationID.fillna(-1, inplace=True)
    answers_dict['Q3 fraction_miss'] = len(df[df.PUlocationID==-1])/len(df)*100
    categorical = ['PUlocationID', 'DOlocationID']
    df[categorical] = df[categorical].astype(str)
    dicts = df[categorical].to_dict(orient='records')
    answers_dict['skipped_rows'] = init_size_df - len(df)
    
    return df, dicts, answers_dict

In [58]:
train_df, train_dicts, answers = _transform(train_df)
test_df, test_dicts, _ = _transform(test_df)

In [59]:
for q, ans in answers.items():
    print(f"{q} - {ans}")

Q2 avg_duration - 19.1672240937939
Q3 fraction_miss - 83.52732770722618
skipped_rows - 44286


In [61]:
dv = DictVectorizer()

X_train = dv.fit_transform(train_dicts)
X_test = dv.transform(test_dicts)

target = 'duration'
y_train = train_df[target].values
y_test = test_df[target].values

### Question 4

In [62]:
print(X_train.shape[1])

525


In [65]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred_train = lr.predict(X_train)

### Question 5

In [66]:
print(mean_squared_error(y_train, y_pred_train, squared=False))

10.528519107223724


### Question 6

In [67]:
y_pred_test = lr.predict(X_test)
print(mean_squared_error(y_test, y_pred_test, squared=False))

11.01428321783417
